# Use the library cirpy
Ref: https://github.com/mcs07/CIRpy

# MY EDIT

In [25]:
import cirpy
import pandas as pd
import numpy as np

# Load your data from the Excel file
data = pd.read_excel("C:\\Users\\ricar\\Documents\MML\\DASS Me TOo Similarity Search\\ppt_df.xlsx", sheet_name="Sheet1")

# Create empty lists to store CAS numbers and SMILES notations
CASes = []
SMILES = []

# Iterate through CAS numbers in the dataframe
for CASRN in data["CASRN"]:
    # Resolve the CAS number to SMILES notation using cirpy
    try:
        smiles = cirpy.resolve(CASRN, 'smiles')
        CASes.append(CASRN)
        SMILES.append(smiles)
    except Exception as e:
        # Handle exceptions if CAS number cannot be resolved
        print(f"Error resolving CAS {CAS}: {str(e)}")

# Create a DataFrame with CAS and SMILES columns
result_df = pd.DataFrame({'CAS': CASes, 'SMILES': SMILES})

# Save the DataFrame to an Excel file
result_df.to_excel("C:\\Users\\ricar\\Documents\MML\\DASS Me TOo Similarity Search\\ppt_df_smiles.xlsx", index=False)


In [6]:
# Get smiles based on chemical CAS number
import cirpy
import pandas as pd

data = pd.read_excel("C:\\Users\\ricar\\Documents\MML\\DASS Me TOo Similarity Search\\ppt_df.xlsx", sheet_name = "Sheet1")
CASes = []
identifiers = data["CAS"].values
for CAS in identifiers:
    CASes += [CAS, cirpy.resolve(CAS, 'smiles')]
New = np.array(CASes).reshape(-1,2)
PD = pd.DataFrame(New)
PD.to_excel("smiles.xlsx")

KeyError: 'CAS'

In [ ]:
## Get smiles based on chemical name
import cirpy
data = pd.read_excel("Data.xlsx", sheet_name = "Sheet1")
names = []
identifiers = data["Name"].values
for name in identifiers:
    names += [name, cirpy.resolve(name, 'smiles')]
New = np.array(names).reshape(-1, 2)
PD = pd.DataFrame(New)
PD.to_excel("smiles.xlsx")

# Use the website http://cactus.nci.nih.gov/

In [ ]:
import pandas as pd
import numpy as np
from urllib.request import urlopen


def CIRconvert(ids):
    try:
        url = 'http://cactus.nci.nih.gov/chemical/structure/' + ids + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        return ans
    except:
        return 'Did not work'


data = pd.read_excel("Data.xlsx", sheet_name="Sheet1")
CASes = []
identifiers = data["CAS"].values
for CAS in identifiers:
    CASes += [CAS,CIRconvert(CAS)]
    New = np.array(CASes).reshape(-1, 2)
    PD = pd.DataFrame(New)
PD.to_excel("smiles.xlsx", index=False)

# Use the website https://echa.europa.eu/

In [ ]:
import selenium.webdriver as webdriver
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import re
from urllib.request import urlopen


def get_results(CAS):
    url = "https://echa.europa.eu/advanced-search-for-chemicals?p_p_id=dissadvancedsearch_WAR_disssearchportlet&p_p_lifecycle=0&p_p_col_id=column-1&p_p_col_count=1"
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    search_box = driver.find_element_by_xpath('//*[@id="_dissadvancedsearch_WAR_disssearchportlet_casNumber"]')
    search_box.send_keys(CAS)
    search_box.submit()
    driver.implicitly_wait(100)
    terms = driver.find_element_by_xpath('//*[@id="_viewsubstances_WAR_echarevsubstanceportlet_acceptDisclaimerButton"]')
    if terms == True:
        ActionChains(driver).move_to_element(terms).click().perform()
        driver.implicitly_wait(100)
    try:
        driver.find_element_by_xpath('//*[@id="_dissadvancedsearch_WAR_disssearchportlet_rmlSearchResultVOsSearchContainerSearchContainer"]/table/tbody/tr[1]/td[4]/a')
        links = driver.find_element_by_xpath('//*[@id="_dissadvancedsearch_WAR_disssearchportlet_rmlSearchResultVOsSearchContainerSearchContainer"]/table/tbody/tr[1]/td[4]/a')
        href = links.get_attribute("href")
        website = urlopen(href).read().decode('utf8')
        if re.search(r"SMILES:</span> </dt> <dd>[^<]+</dd>", website):
            details = re.search(r"SMILES:</span> </dt> <dd>[^<]+</dd>", website)
            details = details.group(0)
            smiles = details[details.find('<dd>') + 4: -5]
            driver.close()
            return smiles
        else:
            return 'No results found'
    except:
        driver.close()
        return 'No results found'


data = pd.read_excel("Data.xlsx", sheet_name = "Sheet1")
CASes = []
identifiers = data["CAS"].values
for CAS in identifiers:
    CASes += [CAS,get_results(CAS)]
    New = np.array(CASes).reshape(-1, 2)
    PD = pd.DataFrame(New)
    PD.to_excel("smiles.xlsx")

# Use the website http://www.ambinter.com/

In [ ]:
from urllib.request import urlopen
from selenium import webdriver

def get_results(CAS):
    url = "http://www.ambinter.com/search/"
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    search_box = driver.find_element_by_id("quick_search_input")
    search_box.send_keys(CAS)
    search_box.submit()
    try:
        links = driver.find_elements_by_xpath('//*[@id="resultArea"]/div[2]/div/table/tbody/tr/td[1]/a')
        href = links[0].get_attribute("href")
        website = urlopen(href).read().decode('utf8')
        details = re.search(r'<th>Smiles</th><td>.*', website)
        details = details.group(0)
        smiles = details[details.find('</th><td>') + 9: -5]
        driver.close()
        return smiles

    except:
        driver.close()
        return 'No results found'


data = pd.read_excel("Data.xlsx", sheet_name = "Sheet1")
CASes = []
identifiers = data["CAS"].values
for CAS in identifiers:
    CASes += [CAS, get_results(CAS)]
    New = np.array(CASes).reshape(-1, 2)
    PD = pd.DataFrame(New)
    PD.to_excel("smiles.xlsx")
